In [15]:
import numpy as np # We'll be storing our data as numpy arrays
import os # For handling directories
from PIL import Image # For handling the images
import matplotlib.pyplot as plt
import matplotlib.image as mpimg # Plotting

lookup = dict()
reverselookup = dict()
count = 0
for j in os.listdir('Train_Final/leapGestRecog/00/'):
    if not j.startswith('.'): # If running this code locally, this is to 
                              # ensure you aren't reading in hidden folders
        lookup[j] = count
        reverselookup[count] = j
        count = count + 1
lookup

{'01_palm': 0,
 '02_l': 1,
 '03_fist': 2,
 '04_fist_moved': 3,
 '05_thumb': 4,
 '06_index': 5,
 '07_ok': 6,
 '08_palm_moved': 7,
 '09_c': 8,
 '10_down': 9}

In [16]:
# -------------------------------------------------------------------------------------------------------
# To prepare dataset including train_set, validation_set, and test_set
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# ImageDataGenerator can create an object used to generate batches of image data
image_datagen = ImageDataGenerator(rescale = 1./255, dtype='float32', validation_split=0.2)

# The ImageDataGenerator class has the method flow_from_directory() 
#       wihch can read the images from folders containing images.
train_set = image_datagen.flow_from_directory(  'Train_Final/leapGestRecog/',
                                                 #target_size = (640, 240),
                                                 target_size = (200, 200),
                                                 batch_size  = 64,
                                                 class_mode  = 'categorical',
                                                 subset='training'
                                                 )

validation_set = image_datagen.flow_from_directory(  'Train_Final/leapGestRecog/',
                                                     #target_size = (640, 240),
                                                     target_size = (200, 200),
                                                     batch_size  = 64,
                                                     class_mode  = 'categorical',
                                                     subset='validation'
                                                     )

test_set = image_datagen.flow_from_directory(   'Test/',
                                                #target_size = (640, 240),
                                                target_size = (200, 200),
                                                batch_size  = 32,
                                                class_mode  = 'categorical',
                                                )


Found 16000 images belonging to 10 classes.
Found 4000 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Concatenate
from tensorflow.keras.applications import VGG16
from keras.models import Sequential,load_model

# Load the pre-trained VGG16 model without the top classification layer
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

# Freeze the pre-trained layers in VGG16
# for layer in vgg16.layers:
#     layer.trainable = False

# # Convert single-channel input to three channels
model = Sequential()
# model.add(tf.keras.layers.Lambda(lambda x: tf.concat([x, x, x], axis=-1), input_shape=(120, 320, 1)))

# Add the VGG16 base model
model.add(vgg16)

# Add custom classification layers on top of VGG16
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
#model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(32, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(16, activation='relu'))
model.add(Dense(10, activation='sigmoid'))
#model.add(Dense(10, activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy',
         optimizer = 'rmsprop',
         metrics = ['accuracy'])

history = model.fit(train_set, epochs = 3, batch_size=64, validation_data=validation_set)

print("Train accuracy:", history.history['accuracy'][-1])
print("Val accuracy:", history.history['val_accuracy'][-1])

print("Train loss:", history.history['loss'][-1])
print("Val loss:", history.history['val_loss'][-1])

test_loss, test_accuracy = model.evaluate(test_set)

print('Test accuracy: {:2.2f}%'.format(test_accuracy*100))

In [ ]:
import lime
import lime.lime_image
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from skimage.segmentation import slic
import matplotlib.pyplot as plt

# Load the pre-trained VGG16 model without the top classification layer
# vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(120, 320, 1))
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

# Freeze the pre-trained layers in VGG16
for layer in vgg16.layers:
    layer.trainable = False

# Create the complete model with VGG16 base
model = Sequential()
model.add(vgg16)
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(10, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

history = model.fit(train_set, epochs=5, batch_size=10, validation_data=validation_set)

# Create an explainer object
explainer = lime.lime_image.LimeImageExplainer()

# Define the prediction function
def predict_fn(images):
    preprocessed_images = preprocess_input(images)
    return model.predict(preprocessed_images)

# Keep track of seen classes and corresponding Lime explanations
seen_classes = set()
lime_explanations = []

# Iterate over the test set and explain a single sample from each class using LIME
for i in range(len(test_set)):
    sample_image = test_set[i][0]
    sample_label = np.argmax(test_set[i][1])
    
    # Check if the current sample belongs to a new class
    if sample_label not in seen_classes:
        # Convert the single-channel grayscale image to three-channel RGB image
        sample_image_rgb = np.repeat(sample_image, 3, axis=-1)
        
        # Reshape the sample_image to match the expected input shape of the model
        sample_image_rgb = np.expand_dims(sample_image_rgb, axis=0)
        
        # Apply superpixel segmentation using SLIC algorithm
        segments = slic(sample_image_rgb[0], n_segments=100, compactness=10, sigma=1)
        
        # Explain the image using LIME
        explanation = explainer.explain_instance(sample_image_rgb[0], predict_fn, top_labels=1, num_samples=1000)
        
        # Add the explanation to the lime_explanations list
        lime_explanations.append((sample_image_rgb[0], explanation))
        
        # Add the class to the seen_classes set
        seen_classes.add(sample_label)
        
        if len(seen_classes) == 10:
            break  # Exit the loop if all 10 classes are seen

# Display the Lime explanations
fig, axes = plt.subplots(10, 2, figsize=(8, 30))

for i, (image, explanation) in enumerate(lime_explanations):
    ax1 = axes[i, 0]
    ax2 = axes[i, 1]
    
    # Show the original image
    ax1.imshow(image)
    ax1.axis('off')
    ax1.set_title('Label: {}'.format(list(seen_classes)[i]))
    
    # Show the Lime explanation
    lime_image, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=5, hide_rest=False)
    ax2.imshow(lime_image)
    ax2.imshow(mask, cmap='jet', alpha=0.7)
    ax2.axis('off')
    ax2.set_title('LIME Explanation')

plt.tight_layout()
plt.show()


In [ ]:
import lime
import lime.lime_image
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from skimage.segmentation import slic
import matplotlib.pyplot as plt

# Load the pre-trained VGG16 model without the top classification layer
# vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(120, 320, 1))
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

# Freeze the pre-trained layers in VGG16
for layer in vgg16.layers:
    layer.trainable = False

# Create the complete model with VGG16 base
model = Sequential()
model.add(vgg16)
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(10, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

history = model.fit(train_set, epochs=3, batch_size=10, validation_data=validation_set)


# Create an explainer object
explainer = lime.lime_image.LimeImageExplainer()

# Define the prediction function
def predict_fn(images):
    return model.predict(images)

# Keep track of seen classes and corresponding Lime explanations
seen_classes = set()
lime_explanations = []

# Iterate over the test set and explain 2 samples from each class using LIME
for i in range(len(test_set)):
    sample_images = test_set[i][0]
    sample_labels = test_set[i][1]
    
    for j in range(len(sample_images)):
        sample_image = sample_images[j]
        sample_label = np.argmax(sample_labels[j])

        # Check if the current sample belongs to a new class or still requires more explanations
        if (len(lime_explanations) < 2) or (sample_label not in seen_classes and len(seen_classes) < 10):
            # Reshape the sample_image to match the expected input shape of the model
            sample_image = np.expand_dims(sample_image, axis=0)
            
            # Apply superpixel segmentation using SLIC algorithm
            segments = slic(sample_image[0], n_segments=100, compactness=10, sigma=1)
            
            # Explain the image using LIME
            explanation = explainer.explain_instance(sample_image[0], predict_fn, top_labels=1, num_samples=1000, segmentation_fn=lambda x: segments)
            
            # Add the explanation to the lime_explanations list
            lime_explanations.append((sample_image[0], explanation))
            
            # Add the class to the seen_classes set
            seen_classes.add(sample_label)
            
        if len(lime_explanations) >= 20:
            break  # Exit the loop if all 20 samples (2 samples from each class) are obtained
            
    if len(lime_explanations) >= 20:
        break  # Exit the outer loop if all 20 samples (2 samples from each class) are obtained

# Display the Lime explanations
fig, axes = plt.subplots(10, 4, figsize=(12, 20))

for i, (image, explanation) in enumerate(lime_explanations):
    ax1 = axes[i // 2, i % 2 * 2]
    ax2 = axes[i // 2, i % 2 * 2 + 1]
    
    # Show the original image
    ax1.imshow(image)
    ax1.axis('off')
    ax1.set_title('Label: {}'.format(list(seen_classes)[i]))
    
    # Show the Lime explanation
    lime_image, mask = explanation.get_image_and_mask(explanation.top_labels[0], positive_only=False, num_features=5, hide_rest=False)
    ax2.imshow(lime_image)
    ax2.imshow(mask, cmap='jet', alpha=0.7)
    ax2.axis('off')
    ax2.set_title('LIME Explanation')

plt.tight_layout()
plt.show()

In [19]:
import lime
import lime.lime_image
import numpy as np
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from skimage.segmentation import slic
import matplotlib.pyplot as plt

# Load the pre-trained VGG16 model without the top classification layer
# vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(120, 320, 1))
vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

# Freeze the pre-trained layers in VGG16
for layer in vgg16.layers:
    layer.trainable = False

# Create the complete model with VGG16 base
model = Sequential()
model.add(vgg16)
model.add(GlobalAveragePooling2D())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(10, activation='sigmoid'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

#history = model.fit(train_set, epochs=3, batch_size=10, validation_data=validation_set)

# Initialize LimeImageExplainer
explainer = lime.lime_image.LimeImageExplainer()

# Choose a random image from your validation set
image = validation_set[0][0]  # Access the first element of the tuple, assuming it contains the image

# Reshape the image to have a single color channel
image = np.squeeze(image)  # Remove any single-dimensional entries from the shape
image = np.expand_dims(image, axis=2)  # Add a single color channel

# Define a function to predict using your model
predict_fn = model.predict(test_set)  # Use 1 channel instead of 3

# Generate an explanation using Lime
explanation = explainer.explain_instance(image, predict_fn, top_labels=1, hide_color=0, num_samples=1000)

# Show the explanation
explanation.show_in_notebook(text=True)




63/63 [==============================] - 5s 78ms/step


ValueError: only 2D color images are supported